# Datová akademie, ČSOB, 2023

---

* [Časové řady](#Časové-řady),
    - [úvodní motivace](#Úvod),
    - [základní objekty](#Základní-objekty),
    - [indexování časovými údaji](#Indexování-s-časem),
    - [funkce to_datetime](#Funkce-TO_DATETIME),
    - [funkce date_range](#Funkce-DATE_RANGE),
    - [resampling](#Resampling,-snížení-frekvence),
    - [zpřesňování](#Zpřesňování-řady),
    - [cvičení](#🧠-CVIČENÍ-🧠,-procvič-si-časové-řady).
* [Výkonné pandy](#Výkonný-PANDAS),
    - [funkce eval](#Funkce-EVAL),
    - [výhody eval](#Výkon-EVAL),
    - [funkce query](#Funkce-QUERY),
    - [souhrn k eval a query](#Souhrn-k-EVAL-a-QUERY),
    - [cvičení](#🧠-CVIČENÍ-🧠,-procvič-si-funkce-EVAL-a-QUERY).
* [Skutečné hodnoty](#Skutečné-datové-sety),
    - [chybějící údaje](#Kolik-mi-chybí-údajů),
    - [encoding](#Encoding),
    - [nekonzistentní data](#Nekonzistentní-data),
    - [cvičení](#🧠-CVIČENÍ-🧠,-procvič-si-práci-s-chybným-datasetem).

<br>

## Časové řady

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

### Úvod

---

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: umožňuje analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny,
* **Efektivní manipulace s časem**: zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny,
* **Flexibilní indexace**: podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat,
* **Kompatibilita s dalšími knihovnami**: je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je `statsmodels` nebo `scikit-learn`.

<br>

#### Pomocí standardního Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [1]:
from datetime import datetime

In [216]:
# help(datetime)

In [3]:
datetime(year=2023, month=4, day=5)

datetime.datetime(2023, 4, 5, 0, 0)

In [4]:
type(datetime(year=2023, month=4, day=5))

datetime.datetime

<br>

Nebo knihovna `dateutil` pro *parsování* datových typů z různých **stringových zadání**:

In [217]:
from dateutil import parser

In [219]:
type(parser)

module

<br>

Funkce `parse` zpracuje (*parsuje*) velké množství různých formátů času ve `str`:

In [8]:
date = parser.parse("5th of april, 2023")

In [221]:
type(parser.parse)

function

In [13]:
date.year

2023

<br>

Kde pomocí metody `strftime` můžeš vypsat den:

In [14]:
date.strftime("%A")

'Wednesday'

In [17]:
date.strftime("%d/%m/%y")

'05/04/23'

<br>

#### Pomocí NUMPY

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [20]:
import numpy
from numpy import array, arange

In [21]:
date = array('2023-04-05', dtype=numpy.datetime64)

In [22]:
date

array('2023-04-05', dtype='datetime64[D]')

<br>

Pokud potřebuješ pole **následujících 7 dní**:

In [23]:
date + arange(7)

array(['2023-04-05', '2023-04-06', '2023-04-07', '2023-04-08',
       '2023-04-09', '2023-04-10', '2023-04-11'], dtype='datetime64[D]')

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji.

Přímo v Pythonu pomocí `datetime` objektu potom roste handicap, když objekty nabývají na velikosti.

<br>

#### Pomocí PANDAS

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [24]:
from pandas import to_datetime

In [25]:
date = to_datetime("5th of April, 2023")

In [26]:
date

Timestamp('2023-04-05 00:00:00')

In [28]:
type(date)

pandas._libs.tslibs.timestamps.Timestamp

In [29]:
date.strftime("%A")

'Wednesday'

<br>

### Základní objekty

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

Tedy práce **s univerzálními datovými typy** pro čas.

Mezi základní objekty pro práci s časem patří:
* `Timestamp`, typ (související struktura Indexu `DatetimeIndex`),
* `Period`, typ (.. `PeriodIndex`),
* `Timedelta`, typ (.. `TimedeltaIndex`).

<br>

#### Timestamp

---

Základní datový typ do časových řad.

Přiřazuje pandasí hodnoty k skutečnému konkrétním časovému okamžiku.

In [30]:
from pandas import Timestamp

In [31]:
Timestamp("2023-05-04")  # str, tuple, datetime.datetime

Timestamp('2023-05-04 00:00:00')

In [32]:
Timestamp(2023, 5, 4)

Timestamp('2023-05-04 00:00:00')

In [34]:
type(Timestamp(2023, 5, 4))

pandas._libs.tslibs.timestamps.Timestamp

<br>

#### Period

---

V mnoha případech je však přirozenější spojit proměnné, jako je **počáteční čas** s následným **časovým rozpětím**.

`Period` představuje časový interval nebo období, které má začátek a *frekvenci*.

In [35]:
from pandas import Period

In [36]:
Period("2023-04", freq="D")

Period('2023-04-01', 'D')

In [37]:
type(Period("2023-04", freq="D"))

pandas._libs.tslibs.period.Period

Hlavním rozdílem pro `Timestamp` a `Period` je v rámci jejich zapracování.

`Timestamp` pracuje s konkrétním časovým okamžikem.

`Period` naopak s časovým intervalem.

<br>

#### Timedelta

---

Představuje rozdíl mezi dvěma datumy nebo časy.

In [229]:
datum_1 = Timestamp(2023,4,12, 12, 30)

In [230]:
datum_2 = Timestamp(2023,4,10, 12, 0)

In [231]:
datum_1 - datum_2

Timedelta('2 days 00:30:00')

<br>

### Indexování s časem

---

Pomocí objektů času můžeš vytvořit samotné **Indexy**.

`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy:

In [38]:
from pandas import DatetimeIndex, Series

<br>

Pomocný objekt typu `list`:

In [39]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [40]:
indexy = DatetimeIndex(datumy)

In [41]:
type(indexy)

pandas.core.indexes.datetimes.DatetimeIndex

In [42]:
jmena_dnu = [to_datetime(den).strftime("%A") for den in datumy]

In [43]:
jmena_dnu

['Wednesday', 'Tuesday', 'Monday', 'Sunday']

In [44]:
df_hodnoty = Series(jmena_dnu, index=indexy)

In [45]:
df_hodnoty

2023-04-05    Wednesday
2022-04-05      Tuesday
2021-04-05       Monday
2020-04-05       Sunday
dtype: object

In [46]:
type(df_hodnoty)

pandas.core.series.Series

In [47]:
type(indexy)

pandas.core.indexes.datetimes.DatetimeIndex

In [48]:
df_hodnoty["2020": "2022"]

/tmp/ipykernel_21015/3590313258.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_hodnoty["2020": "2022"]


2022-04-05    Tuesday
2021-04-05     Monday
2020-04-05     Sunday
dtype: object

In [49]:
df_hodnoty["2020-04-05": "2022-04-05"]

2022-04-05    Tuesday
2021-04-05     Monday
2020-04-05     Sunday
dtype: object

<br>

#### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [96]:
datumy

DatetimeIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-05-04',
               '2023-04-05'],
              dtype='datetime64[ns]', freq=None)

In [95]:
specialni_datumy = to_datetime([
    datetime(2022, 4, 5), "5th of April 2021", "2020-Apr-5", "05-04-2019", "20180405"]
)

In [99]:
datumy - specialni_datumy

TimedeltaIndex(['365 days', '730 days', '1095 days', '1461 days', '1826 days'], dtype='timedelta64[ns]', freq=None)

<br>

### Funkce TO_DATETIME

---

Nejčastější datové typy, které lze zapsat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

In [50]:
from pandas import to_datetime

<br>

#### Práce s jedním datumem

---

In [51]:
datum = to_datetime("05/04/2023")  # str, tuple, datetime.datetime

In [52]:
datum

Timestamp('2023-05-04 00:00:00')

In [53]:
type(datum)

pandas._libs.tslibs.timestamps.Timestamp

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

---

Pokud se parser ve funkci `to_datetime` ztratí nebo tvoje zadání neodpovídá jeho vyhotovení, můžeš zadat formát explicitně:

In [232]:
vystup = to_datetime("12-11-2010 01:05")

In [233]:
vystup.day

11

<br>

V ukázce výš ale hodnota `11` představuje měsíc, nikoliv den.

In [54]:
vystup = to_datetime("12-11-2010 01:05", format="%d-%m-%Y %H:%M")

In [55]:
vystup.day

12

In [56]:
vystup.month

11

In [57]:
vystup.year

2010

In [58]:
vystup.hour

1

<br>

#### Časové zóny

---

Objekt `Timestamp` umožňuje hodnoty časových zón zadávat pomocí metody `tz_localize`:

In [238]:
datum

Timestamp('2023-05-04 00:00:00')

In [240]:
tz_datum = datum.tz_localize("Europe/Prague")

In [241]:
tz_datum.tz

<DstTzInfo 'Europe/Prague' CEST+2:00:00 DST>

In [246]:
nove_tz_datum = tz_datum.tz_convert("US/Pacific")  # Europe/London

In [247]:
nove_tz_datum

Timestamp('2023-05-03 15:00:00-0700', tz='US/Pacific')

<br>

#### Práce s několika datumy

---

Stejně jak umí funkce *parsovat* jednu hodnotu, umí zpracovat pole časových stringů:

In [236]:
datumy = to_datetime(
    [datetime(2023, 4, 5), "5th of April 2023", "2023-Apr-5", "04-05-2023", "20230405"]
)

In [237]:
datumy

DatetimeIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-04-05',
               '2023-04-05'],
              dtype='datetime64[ns]', freq=None)

In [61]:
type(datumy)

pandas.core.indexes.datetimes.DatetimeIndex

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje, vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [64]:
datumy.to_period?

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

#### Denní data

In [65]:
datumy.to_period("D")

PeriodIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-05-04',
             '2023-04-05'],
            dtype='period[D]')

#### Měsíční data

In [66]:
datumy.to_period("M")

PeriodIndex(['2023-04', '2023-04', '2023-04', '2023-05', '2023-04'], dtype='period[M]')

<br>

### Funkce DATE_RANGE

---

In [74]:
from pandas import date_range, DataFrame

In [68]:
date_range(start='2023-01-01', periods=10, freq='D')

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10'],
              dtype='datetime64[ns]', freq='D')

In [70]:
data = {
    'date': date_range(start='2022-01-01', periods=10, freq='D'),
    'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]
}

In [71]:
data

{'date': DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
                '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
                '2022-01-09', '2022-01-10'],
               dtype='datetime64[ns]', freq='D'),
 'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]}

In [75]:
df_data = DataFrame(data)

In [77]:
df_data.head(3)

,date,sales
0,2022-01-01,100
1,2022-01-02,102
2,2022-01-03,105


In [78]:
df_data.set_index('date', inplace=True)

In [79]:
df_data.head(3)

,sales
date,
2022-01-01,100
2022-01-02,102
2022-01-03,105


In [80]:
df_period_y = df_data.to_period(freq='Y')

In [81]:
df_period_y

,sales
date,
2022,100
2022,102
2022,105
2022,107
2022,110
2022,112
2022,115
2022,117
2022,120


<br>

#### Počet period

---

Nachystá objekt, který vypracuje tolik period, které definuješ pomocí parametru `periods`:

In [82]:
date_range(start='2000-01-01', periods=10, freq='Y')

DatetimeIndex(['2000-12-31', '2001-12-31', '2002-12-31', '2003-12-31',
               '2004-12-31', '2005-12-31', '2006-12-31', '2007-12-31',
               '2008-12-31', '2009-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')

In [100]:
datumy_ind = date_range(start="01-01-1992", periods=8)  # defaultně denní frekvence

In [101]:
datumy_ind

DatetimeIndex(['1992-01-01', '1992-01-02', '1992-01-03', '1992-01-04',
               '1992-01-05', '1992-01-06', '1992-01-07', '1992-01-08'],
              dtype='datetime64[ns]', freq='D')

<br>

#### Frekvence

---

In [102]:
datumy_mesicne_ind = date_range("01-01-1992", periods=8, freq="M")

In [103]:
datumy_mesicne_ind

DatetimeIndex(['1992-01-31', '1992-02-29', '1992-03-31', '1992-04-30',
               '1992-05-31', '1992-06-30', '1992-07-31', '1992-08-31'],
              dtype='datetime64[ns]', freq='M')

<br>

#### Hodinové periody

---

In [105]:
from pandas import timedelta_range

In [ ]:
hodinove_ind = timedelta_range(0, periods=12, freq="H")

In [ ]:
hodinove_ind

<br>

### Frekvence

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovníḧo měsíce |
| `BQ` | konec pracovního čtvrtletí |

#### Netradiční frekvence

---

In [108]:
timedelta_range(0, periods=20, freq="1H30T")

TimedeltaIndex(['0 days 00:00:00', '0 days 01:30:00', '0 days 03:00:00',
                '0 days 04:30:00', '0 days 06:00:00', '0 days 07:30:00',
                '0 days 09:00:00', '0 days 10:30:00', '0 days 12:00:00',
                '0 days 13:30:00', '0 days 15:00:00', '0 days 16:30:00',
                '0 days 18:00:00', '0 days 19:30:00', '0 days 21:00:00',
                '0 days 22:30:00', '1 days 00:00:00', '1 days 01:30:00',
                '1 days 03:00:00', '1 days 04:30:00'],
               dtype='timedelta64[ns]', freq='90T')

<br>

### Resampling, snížení frekvence

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje *agregaci* dat.

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [109]:
data = {
    'date': date_range(start='2022-01-01', periods=60, freq='D'),
    'sales': range(60)
}

In [110]:
data

{'date': DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
                '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
                '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12',
                '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16',
                '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20',
                '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24',
                '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28',
                '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01',
                '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05',
                '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09',
                '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13',
                '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17',
                '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21',
                '2022-02-22', '2022-02-23', '2022-02-24'

In [111]:
df_data = DataFrame(data)

In [112]:
df_data.set_index('date', inplace=True)

In [113]:
df_data.head()

,sales
date,
2022-01-01,0
2022-01-02,1
2022-01-03,2
2022-01-04,3
2022-01-05,4


In [120]:
mesicni_data = df_data.resample('M').sum()

In [121]:
mesicni_data

,sales
date,
2022-01-31,465
2022-02-28,1246
2022-03-31,59


<br>

### Zpřesňování řady

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr novou frekvenci, na kterou chcete data převést.

In [122]:
data = {
    'date': date_range(start='2022-01-01', end='2022-03-01', freq='MS'),  # M - konec měsíce
    'sales': [100, 120, 150]
}

In [123]:
df_data = DataFrame(data)

In [124]:
df_data.head()

,date,sales
0,2022-01-01,100
1,2022-02-01,120
2,2022-03-01,150


In [125]:
df_data.set_index('date', inplace=True)

In [126]:
df_data.head()

,sales
date,
2022-01-01,100
2022-02-01,120
2022-03-01,150


In [134]:
denni_data = df_data.asfreq('D', method='ffill')

In [135]:
denni_data.head()

,sales
date,
2022-01-01,100
2022-01-02,100
2022-01-03,100
2022-01-04,100
2022-01-05,100


Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `D`, což znamená **denní data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

<br>

##### **🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [ ]:
import pandas as pd
import numpy as np

denni_datumy = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
prodeje = np.random.randint(10, 100, size=(len(date_rng),))

data = {
    'datumy': denni_datumy,
    'prodeje': prodeje
}

df_prodeje = pd.DataFrame(data)
df_prodeje.set_index('datumy', inplace=True)

In [139]:
df_prodeje.tail()

,prodeje
datumy,
2022-12-27,10
2022-12-28,28
2022-12-29,80
2022-12-30,12
2022-12-31,90


<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    mesicni_prodej = df_prodeje.resample('M').sum()

    df_prodeje['weekday'] = df_prodeje.index.weekday
    denni_prumer = df_prodeje.groupby('weekday')['prodeje'].mean()

    prodej_posledni_vikend = df_prodeje.loc['2022-12-25':, 'prodeje']
    ```
</details>

## Výkonný PANDAS

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.riOR-y2_yu_2cGdEN2e6-wAAAA%26pid%3DApi&f=1&ipt=638c4516d155ff9768066e17887a621cbad0de6bce639a40be98f5dedbce67ac&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Z předchozích ukázek a popisků (seskupování) tedy můžeš chápat, že skutečný výkon *frameworku* tkví v převedení **základních operací do jazyka C**.

Tyto abstrakce jsou účinné a efektivní pro mnoho běžných případů použití.

Vytvářejí ale spousty dočasných objektů, což může způsobit nepřiměřenou režii výpočetního času a paměti.

<br>

### EVAL a QUERY

---

Přímý přístup k operacím rychlosti C bez nákladné alokace `pandas` poskytuje funkce `eval` a `query`.

Ty spolehájí na funkci `numexpr`.

<br>

### Funkce EVAL

---

Dva důvody, proč používat funkci `eval` místo obyčejného Pythonu:
1. velké objekty typu `DataFrame` jsou vyhodnocovány efektivněji,
2. aritmetické a logické výrazy projdou naráz základním enginem (pomocí `numexpr`).

**Pozor!** Dobrá obecná pomůcka říká, že není nutné využívat funkci `eval` pro tabulky, které jsou kratší než 10 000 řádků. 

### Použití EVAL

---



Funkce `eval` slouží k efektivnímu vyhodnocení **aritmetických výrazů** na:
* objekty `DataFrame`,
* objekty `Series`.

Její největší výhodou je **rychlost a efektivita**, která spočívá v minimalizaci paměťové zátěže a zrychlení operací.

Když pracuješ **s velkými datovými sadami**, paměťová zátěž a rychlost se stávají klíčovými faktory.

Funkce `eval` optimalizuje výpočetní proces tím, že snižuje množství alokované paměti a zrychluje výpočetní operace.

<br>

### Ukázka EVAL

---

Pro zadaný *dummy dataset* vytvoř **nový sloupeček** `sloupec_D`, podle vzoru:
`A + B * C`

In [140]:
import pandas as pd

In [141]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

In [143]:
df_data = pd.DataFrame(data)

In [144]:
df_data.head()

,A,B,C
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


In [145]:
df_data['sloupec_D'] = df_data['A'] + df_data['B'] * df_data['C']

In [146]:
df_data.head()

,A,B,C,sloupec_D
0,1,6,11,67
1,2,7,12,86
2,3,8,13,107
3,4,9,14,130
4,5,10,15,155


<br>

*Alternativní způsob*, jak provést stejný výpočet, pokud máš dlouhou tabulku nebo sloupeček, je použití funkce `eval`:

In [147]:
df_data['sloupecek_D'] = df_data.eval('A + B * C')

In [148]:
df_data.head()

,A,B,C,sloupec_D,sloupecek_D
0,1,6,11,67,67
1,2,7,12,86,86
2,3,8,13,107,107
3,4,9,14,130,130
4,5,10,15,155,155


<br>

Funkce `eval` zpracovává zadaný `str` podobně jako built-in funkce `eval`.

Souhrnně lze říci:
* funkci `eval` použij tehdy, pokud je [dataset dlouhý](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html#pandas-eval-performance),
* ve všech ostatních scénářích, kde nepotřebuješ šetřit pamět a čas použij klasickou syntaxi.

### Výkon EVAL

---

In [3]:
import numpy as np
import pandas as pd

In [4]:
pocet_radku, pocet_sloupcu = 1_000_000, 100

In [5]:
rng = np.random.RandomState()

In [6]:
dframe_1, dframe_2, dframe_3, dframe_4 = [DataFrame(rng.rand(pocet_radku, pocet_sloupcu)) for _ in range(4)]

In [7]:
dframe_1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.573963,0.496790,0.844526,0.338041,0.499414,0.164377,0.575126,0.252999,0.850244,0.076448,...,0.454482,0.596613,0.710351,0.146759,0.500802,0.102248,0.582700,0.028887,0.750315,0.733113
1,0.624119,0.327659,0.718025,0.793555,0.499811,0.494495,0.866891,0.992220,0.722946,0.519524,...,0.925290,0.569156,0.153587,0.015681,0.272400,0.865552,0.379229,0.806783,0.934514,0.935086
2,0.033765,0.109566,0.564036,0.904380,0.009347,0.209880,0.446216,0.592211,0.705765,0.285138,...,0.575510,0.434943,0.606159,0.043206,0.485704,0.988841,0.239451,0.893894,0.112089,0.156187
3,0.506136,0.292988,0.012344,0.864872,0.725185,0.070574,0.780162,0.651835,0.302322,0.604492,...,0.415523,0.933595,0.985697,0.501816,0.734839,0.382540,0.312869,0.940625,0.718645,0.122026
4,0.581624,0.769404,0.831302,0.338512,0.997616,0.028020,0.510785,0.999653,0.054756,0.996700,...,0.795238,0.625561,0.761913,0.765882,0.525977,0.148989,0.552476,0.334133,0.895043,0.032326


#### Ukázka sčítání hodnot tabulek

---

In [2]:
from pandas import DataFrame

In [257]:
df1 = DataFrame({"A": [1, 2], "B": [3, 4]})

In [258]:
df2 = DataFrame({"A": [2, 3], "B": [1, 2]})

In [259]:
df1 + df2

,A,B
0,3,4
1,5,6


In [154]:
%timeit dframe_1 + dframe_2 + dframe_3 + dframe_4

1.41 s ± 260 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [155]:
%timeit pd.eval("dframe_1 + dframe_2 + dframe_3 + dframe_4")

1.28 s ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [157]:
# pd.eval?

<br>

### Funkce QUERY

---

Funkce `query` slouží k efektivnímu **filtrování řádků**.

Filtrování probíhá na základě **zadané podmínky**.


Hlavní výhodou jsou:
* **snadná čitelnost**, umožňuje zapisovat podmínky ve snadno čitelné formě, zejména v případě složitějších dotazů,
* **rychlost**, může být rychlejší než tradiční metody filtrování,(zejména pro velké datové sady).

### Ukázka QUERY

---

Pro zadaný *dummy dataset* vyfiltruj takové záznamy, kde:
1. Hodnoty ze sloupce A budou **menší než 4**,
2. hodnoty ze sloupce B budou **větší než 6**,
3. hodnoty ze sloupce C budou **větší než 11**.

In [158]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15],
        'D': ["AA", "BB", "CC", "DD", "EE"]
}

In [159]:
df_data = DataFrame(data)

In [160]:
df_data

,A,B,C,D
0,1,6,11,AA
1,2,7,12,BB
2,3,8,13,CC
3,4,9,14,DD
4,5,10,15,EE


In [161]:
df_filtrovane = df_data[(df_data['A'] < 4) & (df_data['B'] > 6) & (df_data['C'] > 11)]  # &, |

In [162]:
df_filtrovane

,A,B,C,D
1,2,7,12,BB
2,3,8,13,CC


Zapisování pomocí *boolean indexování* je ovšem náročné jak pro zápis, tak pro čtení.

<br>

Pomocí funkce `query`:

In [163]:
df_filtrovane = df_data.query('A < 4 and B > 6 and C > 11')

In [164]:
df_filtrovane

,A,B,C,D
1,2,7,12,BB
2,3,8,13,CC


#### WHERE name LIKE 'CC'

---

In [169]:
df_filtrovane_str = df_data.query(
    "A < 4 and B > 6 and C > 11 and ~D.str.contains('CC')"
)  # LIKE/ NOT LIKE

In [170]:
df_filtrovane_str

,A,B,C,D
1,2,7,12,BB


#### column_a NOT IN []

---

In [ ]:
"A not in [1, 2]"

<br>

### Nedostatky QUERY

---

1. **Omezená syntaxe**, má omezenější syntaxi ve srovnání s běžným zápisem v Pythonu a nemusí podporovat [všechny operace](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html#eval-examples),
2. **Názvy sloupců**, pokud názvy sloupců obsahují **mezery nebo znaky**, musíš je uvést pomocí backticků a použít symbol `@` pro reference na proměnné mimo `DataFrame`.

In [8]:
df_cisla = DataFrame({'Sloupec 1': [1, 2, 3, 4, 5], 'Sloupec 2': [10, 20, 30, 40, 50]})

In [9]:
df_cisla

,Sloupec 1,Sloupec 2
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [174]:
# vystup_1 = df_cisla.query('Sloupec 1 < 3')

In [13]:
limit = 3

In [14]:
vystup_2 = df_cisla.query('`Sloupec 1` < @limit')

In [15]:
vystup_2

,Sloupec 1,Sloupec 2
0,1,10
1,2,20


### Souhrn k EVAL a QUERY

---

Hlavním účelem těchto funkcí je aplikace na skutečně **velké datové sety**.

Rozdíly u drobnějších datasetů **jsou často minimální**.

Velký vliv na výstupný čas dělá parametr `engine`, který se snaží pracovat s knihovnou `numexpr`.

<br>

#### **🧠 CVIČENÍ 🧠, procvič si funkce EVAL a QUERY**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Vyber studenty s věkem **mezi 20 a 22 lety (včetně)** a email, který obsahuje string **gmail.com**,
2. vypočtěte *BMI* (Body Mass Index) pro každého studenta a přidejte ho do nového sloupce (vzorec: *váha[kg] / (výška[m] * výška[m])*)
3. vyberte studenty s BMI vyšším než 22 a kteří pochází z měst Brno a Praha.

In [ ]:
uzivatele = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [19, 20, 21, 22, 23],
    'Height_cm': [160, 170, 175, 180, 165],
    'Weight_kg': [50, 65, 70, 80, 55],
    'Email': ["alice@gmail.com", "bob@tech-users.com", "charlie91@mail.com", "david_bB@gmail.com", "eevee@my_domain.net"],
    'Address': [
        "Vrbová 215/14, 602 00, Brno, Česká republika",
        "Jilmová 862, 400 11, Ústí nad Labem, Česká republika",
        "Kamenická 37/2, 170 00, Praha, Česká republika",
        "Podlipná 1284/6, 460 01, Liberec, Česká republika",
        "Havlíčkova 1703, 500 02, Hradec Králové, Česká republika"
    ]
}

In [ ]:
df_uzivatele = DataFrame(uzivatele)

In [ ]:
df_uzivatele

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    vyber_uzivatel = df_uzivatele.query("20 <= Age <= 22 and Email.str.contains('gmail.com')")

    df_uzivatele['Height_m'] = df_uzivatele.eval('Height_cm / 100')
    df_uzivatele['BMI'] = df_uzivatele.eval('Weight_kg / (Height_m * Height_m)').round(1)
    
    df_prodeje['weekday'] = df_prodeje.index.weekday
    average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
    
    df_uzivatele["City"] = df_uzivatele["Address"].apply(lambda x: x.split(",")[2])
    students_with_high_bmi = df_uzivatele.query('BMI > 22 and City in ["Brno", "Praha"]')
    ```
</details>

<br>

## Skutečné datové sety

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JJu86kKRph1LHt5M7agODQHaHa%26pid%3DApi&f=1&ipt=5cfb9e9f7b451b3a4f7a3ebdc4eb99177d477f20f8145cd818cc4966143f8b2b&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Čištění dat je proces, který patří k samotné práci s daty.

Spolu s některými souvisejícími úkony, patří mezi ty více frustrující.

Nesmyslné datové typy, zkomolené časové údaje, nefungující transformace.

In [177]:
df_uzivatele = DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [None, 20, None, 22, 23],
    'Height_cm': [160, 170, 175, None, 165],
    'Weight_kg': [50, 65, 70, 80, 55]
})

In [178]:
df_uzivatele.head(3)

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70


Ihned po načtení je nejlepší data prozkoumat.

Nejenom datové typy ale také mít povědomí **o chybějících hodnotách**:

In [179]:
df_uzivatele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       5 non-null      object 
 1   Age        3 non-null      float64
 2   Height_cm  4 non-null      float64
 3   Weight_kg  5 non-null      int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 288.0+ bytes


<br>

Pomocí metody `info` si uděláš aspoň povrchní přehled o hodnotách, datových typech a chybějících hodnotách.

<br>

### Kolik mi chybí údajů

---

Vždycky přistupuj k datům s mírnou skepsí.

Otázkou potom není, *JESTLI* mi chybí data, ale *KOLIK* dat mi chybí **a kde**.

In [180]:
chybejici_hodnoty = df_uzivatele.isnull()  # isna()

In [181]:
chybejici_hodnoty

,Name,Age,Height_cm,Weight_kg
0,False,True,False,False
1,False,False,False,False
2,False,True,False,False
3,False,False,True,False
4,False,False,False,False


<br>

Pomocí metody `isnull` nahradíš všechny chybějící hodnoty v tabulce/sloupcích pomocí `True`.

Pokud nechybí, nahradí s `False`.

In [182]:
type(chybejici_hodnoty)

pandas.core.frame.DataFrame

Takový výstup ale není zcela reprezentativní.

Proto je potřeba, sečíst všechny `True` hodnoty po sloupcích:

In [183]:
chybejici_hodnoty = df_uzivatele.isnull().sum()

In [184]:
chybejici_hodnoty

Name         0
Age          2
Height_cm    1
Weight_kg    0
dtype: int64

<br>

Opět v jednotkách to nemusí zcela prozrazovat chybovost.

Nejlepším indikátorem ale bývají procenta z celkového množství hodnot ve sloupci:

In [185]:
celkem_zaznamu = df_uzivatele.shape[0]  # 0 ... Indexy, 1 ... sloupce

In [186]:
celkem_chybi = chybejici_hodnoty

In [187]:
v_procentech = round(celkem_chybi / celkem_zaznamu * 100, 1)

In [188]:
v_procentech

Name          0.0
Age          40.0
Height_cm    20.0
Weight_kg     0.0
dtype: float64

<br>

případně celkové množství záznamů ve všech sloupcích:

In [189]:
vsechny_bunky = df_uzivatele.shape[0] * df_uzivatele.shape[1]  # 5 * 4

In [190]:
vsechny_chybejici_hodnoty = chybejici_hodnoty.sum()            # 4

In [191]:
celkem_v_procentech = round(vsechny_chybejici_hodnoty / vsechny_bunky * 100, 1)

In [192]:
celkem_v_procentech

15.0

<br>

V této ukázce tedy chybí 20 % dat a to není málo!

<br>

### Co s chybějícími hodnotami

---

Je důležité vědět, proč máš takové množství chybějících hodnot.

Chybí ti data, protože neexistují, nebo protože nejsou součástí záznamů v datasetu.

V takových krocích je obvykle nutné, pochopit podstatu údajů (dokumentace, specifikace,..).

Nejjednodušší způsoby, jak s chybějícími daty naložit (ale ne efektivní!!):
1. Zahodit chybějící hodnoty,
2. doplnit chybějící hodnoty.

#### Zahodit chybějící hodnoty

In [193]:
df_uzivatele.dropna()  # axis=0

,Name,Age,Height_cm,Weight_kg
1,Bob,20.0,170.0,65
4,Eve,23.0,165.0,55


<br>

V tento okamžik si zahodíš všechny záznamy, které **postrádaly nějakou hodnotu**.

Někdy je jednodušší zahodit sloupeček, který má chybějící hodnotu:

In [194]:
df_uzivatele.dropna(axis=1)

,Name,Weight_kg
0,Alice,50
1,Bob,65
2,Charlie,70
3,David,80
4,Eve,55


#### Doplnit automaticky hodnoty

In [195]:
df_uzivatele.fillna(0)  # replace: "Nan" --> 0

,Name,Age,Height_cm,Weight_kg
0,Alice,0.0,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,0.0,175.0,70
3,David,22.0,0.0,80
4,Eve,23.0,165.0,55


<br>

Pomocí funkce `fillna` můžeš vyplnit místo chybějících hodnot předdefinovanou vlastní hodnotu.

Nebo pomocí argumentu pro `method` pracovat s elegantnějším zadáním.

Nahradit chybějící hodnoty hodnotou, která následuje bezprostředně za ní ve stejném sloupci (To má velký smysl u souborů dat, kde mají pozorování nějaké logické pořadí.)

In [196]:
df_uzivatele.fillna(method='bfill', axis=0)

,Name,Age,Height_cm,Weight_kg
0,Alice,20.0,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,22.0,175.0,70
3,David,22.0,165.0,80
4,Eve,23.0,165.0,55


## Encoding

---

Problémy s kódováním jsou běžné.

Jde o proces, který mapuje bajtové stringy (`0110101011`) na uživatelsky čitelný text (`"ahoj!"`).

Tento problém nastane, pokud se snažíš načíst zdroj v jiné kódovací sadě, než byl soubor zapsaný.

<br>

Protože je těchto sad hodně, občas skončíš s escapovanými znaky, nebo s neznámými kliky-háky:

In [ ]:
æ–‡å

<br>

Případně pokud nelze domapovat znaky:

��

<br>

V Pythonu 2 nebylo lehké encoding zajistit.

V Pythonu 3 je celý proces o dost jednodušší.

Standardem pro práci bývá obyčejně UTF-8. Na to ale zrovna v našich končinách nemůžeš spoléhat.

In [197]:
veta = "Matouš zaplatil 100 $"

In [199]:
# veta?

<br>

Údaj, který vidíš výš je datový typ `str`.

In [200]:
prevedena_veta = veta.encode("utf-8", errors="replace")

In [201]:
prevedena_veta

b'Matou\xc5\xa1 zaplatil 100 $'

In [ ]:
prevedena_veta?

<br>

Dál je možné, převést `str` na `bytes`. Tedy sekvenci čísel.

Jak ale teď tato sekvence vypadá:

In [ ]:
prevedena_veta

<br>

Celou větu můžeš zpátky kódovat pomocí funkce `decode` z `bytes` na `str`:

In [202]:
print(prevedena_veta.decode("utf-8"))

Matouš zaplatil 100 $


<br>

Pokud ale zvolíš jinou sadu, nemusíš dostat stejné hodnoty:

In [203]:
print(prevedena_veta.decode("windows-1250"))

MatouĹˇ zaplatil 100 $


In [204]:
print(prevedena_veta.decode("ascii"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 5: ordinal not in range(128)

<br>

Toto a další neúspěšné postupy je samozřejmě špatně a ty se tomu chceš určitě vyhnout.

In [205]:
from pandas import read_csv

In [207]:
# chybna_sada = read_csv("neznamy_vzorek.csv")

<br>

Ne vždy je možné, defaultně pracovat se znakovou sadou **UTF-8**.

Ve výstupu je vidět, že jde o směs několika sad a ty musíš vyzkoušet, jaká bude fungovat.

In [208]:
import sys

In [209]:
sys.getdefaultencoding()

'utf-8'

In [ ]:
!pip install chardet

In [210]:
from chardet.universaldetector import UniversalDetector

In [211]:
detector = UniversalDetector()

In [212]:
for line in open("neznamy_vzorek.csv", 'rb'):
    detector.feed(line)
    if detector.done:
        break

detector.close()
print(detector.result)

{'encoding': 'Windows-1252', 'confidence': 0.7299788775910958, 'language': ''}


<br>

Pomocí knihovny `chardet` / `charset_normalizer` identifikuješ, o jaké znakové sady jde:

Nyní máš lepší představu o tom, jakou znakovou sadu soubor používá.

Opatrně na délku `str`, který detekuješ.

Pokud je příliš krátký, může to ovlivnit výsledek.

Naopak pokud je příliš dlouhý, může trvat jeho načtení.

In [213]:
spravna_sada = read_csv("neznamy_vzorek.csv", encoding="Windows-1252")

/tmp/ipykernel_21015/3584548271.py:1: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  spravna_sada = read_csv("neznamy_vzorek.csv", encoding="Windows-1252")


In [214]:
spravna_sada.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


In [ ]:
spravna_sada.info()

<br>

Tentokrát soubor otevřeš pouze s varováním.

Pro příště je určitě výhodou vytvořit kopii takového souboru, kterou uložíš v ideálním kódování:

##### spravna_sada.to_csv("kopie_sada_utf8.csv", encoding="utf-8")

In [ ]:
pokus = read_csv("vzorek_2.csv")

In [ ]:
for line in open("vzorek_2.csv", 'rb'):
    detector.feed(line)
    if detector.done:
        break

detector.close()
print(detector.result)

In [ ]:
pokus_2 = read_csv("vzorek_2.csv", encoding="Windows-1252", delimiter=";")

In [ ]:
pokus_2.head(6)

<br>

## Nekonzistentní data

---

Pokud ti data nechybí, ještě neznamená, že musí být nutně v pořádku:

In [ ]:
from pandas import DataFrame

In [ ]:
chybny_dataset = {
    "id": [111, 112, 113, 114, 115, 116],
    "jmeno": ["Matous", "Marek", "Petr", "Filip", "Jan", "Lukas"],
    "vek": [22, 29, 31, 55, 43, 61],
    "zeme": ["Ceska republika", "Slovensko", "Nemecko", "Ceskarepublika", "Ceska republika", "Rakouskou"]
}

In [ ]:
df_zamestnanci = DataFrame(chybny_dataset)

In [ ]:
df_zamestnanci.head()

Různé zdroje, správci můžou způsobit nejednotný zápis a dopustit se *nekonzistence*.

In [ ]:
df_zamestnanci["zeme"].unique()

<br>

To můžou být jak malá velká písmena, tak různé znaky, chybějící mezery apod.

Odstranit je není náročné.

Náročné může být opět rozpoznání, takové komplikace.

In [ ]:
df_zamestnanci["zeme"].str.lower()

<br>

Některé chyby, ale můžou způsobit paseku. Třeba chybějící mezery.

Tady je nejlepší, pomoci si vhodným nástrojem `fuzzywuzzy`.

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import process

Pro menší datasety s chybějícími mezerami aj., může tato knihovna pracovat prakticky sama.

Účel této knihovny je rozpoznat podobné řetězce, jaké

In [ ]:
shodne = fuzzywuzzy.process.extract(
    "Ceska republika",
    df_zamestnanci["zeme"],
    limit=5,
    scorer=fuzzywuzzy.fuzz.token_sort_ratio
)

In [ ]:
shodne

Nyní můžeš vidět shodné `str` se zvoleným zadáním, jak to vidí `fuzzywuzzy`.

Pro nahrazení je optimální nachystat vhodnou uživatelskou funkci:

In [ ]:
def nahrad_shody_stringem(dframe, sloupec, vzor, min_shoda=60):
    vsechny_stringy = dframe[sloupec].unique()
    
    shody = fuzzywuzzy.process.extract(vzor, vsechny_stringy, limit=5,
                                       scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    nejblizsi_shoda = [shoda [0] for shoda in shody if shoda[1] >= min_shoda]
    
    shodujici_zaznam = dframe[sloupec].isin(nejblizsi_shoda)
    dframe.loc[shodujici_zaznam, sloupec] = vzor

In [ ]:
nahrad_shody_stringem(dframe=df_zamestnanci, sloupec="zeme", vzor="Ceska republika")

In [ ]:
df_zamestnanci

<br>

#### **🧠 CVIČENÍ 🧠, procvič si práci s chybným datasetem**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Nahraj soubor do `DataFrame`, jméno souboru `cviceni_3.csv` (správný encoding, správné rozdělení),
2. projdi sloupce, zkontroluj, kde je problém s daty,
3. vypiš všechny záznamy, které mají ve sloupci `mesto` hodnotu `Praha`.

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    df_ukazka = read_csv("cviceni_3.csv", encoding="Windows-1250", delimiter=";")

    def nahrad_shody_stringem(df, vzor, vyber, min_shoda=85):
        shody = fuzzywuzzy.process.extract(vzor, vyber, limit=20,
                                           scorer=fuzzywuzzy.fuzz.token_sort_ratio)

        nejblizsi_shoda = [shoda[0] for shoda in shody if shoda[1] >= min_shoda]

        shodujici_zaznam = vyber.isin(nejblizsi_shoda)
        df.loc[shodujici_zaznam, "mesto"] = vzor
        
    nahrad_shody_stringem(df_ukazka, "Praha", df_ukazka["mesto"])
    vystup = df_ukazka[df_ukazka["mesto"] == "Praha"]
    ```
</details>

---